Run the DIVAnd analysis, using DIVAnd fithorzlen() to estimate the correlation length input parameters from the observations. Use the domain of the observations as the input domain for the analysis (not the domain of the mask).

In [1]:
using DIVAnd
using CSV
using NCDatasets
using DataFrames
using Statistics

In [3]:
# To run in command prompt or something
# julia compute_DIVAnd_pmn.jl

var_name = "Oxy"
standard_depth = 0
year = 1991
szn = "AMJ"
# subsamp_interval = 1

# obs_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
#     "value_vs_depth\\14_sep_by_sl_and_year\\")

# mask_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
#     "value_vs_depth\\16_diva_analysis\\masks\\")

# pmn_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
#     "value_vs_depth\\16_diva_analysis\\pmn\\")

# output_dir = string("C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\",
#     "value_vs_depth\\16_diva_analysis\\")

# Paths for Linux:
obs_dir = string("/home/hourstonh/Documents/climatology/data/",
    "value_vs_depth/14_sep_by_sl_and_year/")

mask_dir = string("/home/hourstonh/Documents/climatology/data/",
    "value_vs_depth/16_diva_analysis/masks/")

pmn_dir = string("/home/hourstonh/Documents/climatology/data/",
    "value_vs_depth/16_diva_analysis/pmn/")

# years = [1991:1:2020;]
# println(years)
# szns = ["JFM", "AMJ", "JAS", "OND"]

"/home/hourstonh/Documents/climatology/data/value_vs_depth/16_diva_analysis/pmn/"

In [4]:
# Open the required files

# Read in standard level data file
obs_filename = string(obs_dir, var_name, "_", standard_depth, "m_", year, 
    "_", szn, ".csv")
    
println(obs_filename)
    
# Pipe operator to dataframe
obs_df = CSV.File(obs_filename) |> DataFrame
    
if size(obs_df)[1] == 0
    println("DataFrame empty -- skip")
end
    
xobs = obs_df[!, :Longitude]
yobs = obs_df[!, :Latitude]
vobs = obs_df[!, :SL_value]

# Compute anomaly field
vmean = mean(vobs)
vanom = vobs .- vmean;

/home/hourstonh/Documents/climatology/data/value_vs_depth/14_sep_by_sl_and_year/Oxy_0m_1991_AMJ.csv


In [5]:
# Set correlation length from fithorzlen
fitcor_dir = "/home/hourstonh/Documents/climatology/diva_explore/correlation_length/"

fitcor_filename = string(fitcor_dir, "Oxy_fithorzlen_mean_lenxy_100m.csv")

fitcor_df = CSV.File(fitcor_filename) |> DataFrame

year_rownum = year - 1990
fitcor_lenxy = fitcor_df[year_rownum, szn]

5.5811005

In [6]:
# Read in mask
mask_filename = string(mask_dir, var_name, "_", standard_depth, "m_", 
    year, "_", szn, "_mask_6min.nc")
    
mask_ds = Dataset(mask_filename)
    
mask = mask_ds["mask"][:,:]  # [1:subsamp_interval:end, 1:subsamp_interval:end]
mask = Bool.(mask)
#     println(typeof(mask))

# Open the pmn netCDF file
pmn_filename = string(pmn_dir, "divand_pmn_for_mask_6min.nc")

pmn_ds = Dataset(pmn_filename)

# Get the inverse of the resolution of the grid
pm = convert(Array{Float64}, pmn_ds["pm"][:,:])
pn = convert(Array{Float64}, pmn_ds["pn"][:,:])

# Get the 2d mesh grid longitude and latitude
Lon2d, Lat2d = ndgrid(mask_ds["lon"][:], mask_ds["lat"][:])

close(mask_ds)
close(pmn_ds)

closed NetCDF NCDataset

In [7]:
# Set some more parameters

signal_to_noise_ratio = 50.  # Default from Lu ODV session
epsilon2_guess = 1/signal_to_noise_ratio  # 1.

# Choose number of testing points around the current value of L (corlen)
nl = 1

# Choose number of testing points around the current value of epsilon2
ne = 1

# Choose cross-validation method
# 1: full CV; 2: sampled CV; 3: GCV; 0: automatic choice between the three
method = 3

3

In [8]:
# # Run the cross-validation
# bestfactorl, bestfactore, cvval, cvvalues, x2Ddata, y2Ddata, cvinter, xi2D, yi2D = DIVAnd_cv(
#     mask, (pm, pn), (Lon2d, Lat2d), (xobs, yobs), vanom, (fitcor_lenxy, fitcor_lenxy), 
#     epsilon2_guess, nl, ne, method)

LoadError: InterruptException:

In [ ]:
# # Apply factors to guesses
# new_lenx = bestfactorl .* lenx_guess
# new_leny = bestfactorl .* leny_guess
# new_epsilon2 = bestfactore .* epsilon2_guess

# old_params = [lenx_guess, leny_guess, epsilon2_guess]
# new_params = [new_lenx, new_leny, new_epsilon2]

# for i=[1,2,3]
#     println("lenx:", old_params[i], "->", new_params[i])
# end

In [9]:
# Run the analysis
va = DIVAndrunfi(mask, (pm, pn), (Lon2d, Lat2d), (xobs, yobs), vanom,
                 (fitcor_lenxy, fitcor_lenxy), epsilon2_guess)

# Add the output anomaly back to the mean of the observations
vout = vmean .+ va

LoadError: OutOfMemoryError()

In [ ]:
# Export vout as a netCDF file in the same dims as the mask
vout_filename = string(output_dir, var_name, "_", standard_depth, "m_",
    year, "_", szn, "_analysis2d.nc")

vout_ds = Dataset(vout_filename, "c")

# Define lat and lon dims
lon_dim_vout = defDim(vout_ds, "lon", length(mask_ds["lon"][1:subsamp_interval:end]))
lat_dim_vout = defDim(vout_ds, "lat", length(mask_ds["lat"][1:subsamp_interval:end]))

# How to add data to the dims?
lon_dim_vout[:] = mask_ds["lon"][1:subsamp_interval:end]
lat_dim_vout[:] = mask_ds["lat"][1:subsamp_interval:end]

vout_var = defVar(vout_ds, "vout", Float32, ("lon", "lat"))
vout_var[:,:] = vout

println(vout_ds)

close(vout_ds)